In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,25608
2,Huelva,Confirmados PDIA 14 días,658
3,Huelva,Tasa PDIA 14 días,"128,22261628700042"
4,Huelva,Confirmados PDIA 7 días,359
5,Huelva,Total Confirmados,25809
6,Huelva,Curados,20844
7,Huelva,Fallecidos,351


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  25608.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6707.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2820 personas en los últimos 7 días 

Un positivo PCR cada 1598 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,25608.0,359.0,658.0,513170.0,69.957324,128.222616,272.0
Condado-Campiña,7447.0,232.0,403.0,156231.0,148.498057,257.951367,172.0
Huelva-Costa,15407.0,103.0,215.0,289548.0,35.572686,74.253664,81.0
Almonte,1020.0,61.0,126.0,24507.0,248.908475,514.138817,44.0
Huelva (capital),6707.0,51.0,90.0,143837.0,35.456802,62.570827,43.0
Bollullos Par del Condado,680.0,37.0,49.0,14387.0,257.176618,340.585251,28.0
Rociana del Condado,374.0,28.0,56.0,7939.0,352.689256,705.378511,19.0
Cartaya,1012.0,22.0,41.0,20083.0,109.545387,204.152766,18.0
Moguer,889.0,22.0,45.0,21867.0,100.608222,205.789546,16.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rociana del Condado,374.0,28.0,56.0,7939.0,352.689256,705.378511,19.0
San Bartolomé de la Torre,234.0,17.0,24.0,3761.0,452.007445,638.128157,14.0
Alosno,150.0,3.0,25.0,3933.0,76.277651,635.647089,2.0
Almonte,1020.0,61.0,126.0,24507.0,248.908475,514.138817,44.0
Niebla,175.0,12.0,17.0,4116.0,291.545190,413.022352,8.0
Bollullos Par del Condado,680.0,37.0,49.0,14387.0,257.176618,340.585251,28.0
Condado-Campiña,7447.0,232.0,403.0,156231.0,148.498057,257.951367,172.0
Paterna del Campo,120.0,5.0,8.0,3457.0,144.634076,231.414521,2.0
Villalba del Alcor,462.0,3.0,7.0,3366.0,89.126560,207.961973,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,150.0,3.0,25.0,3933.0,76.277651,635.647089,2.0,0.120000
Isla Cristina,2011.0,2.0,12.0,21393.0,9.348852,56.093115,3.0,0.166667
Valverde del Camino,429.0,3.0,9.0,12750.0,23.529412,70.588235,2.0,0.333333
San Juan del Puerto,372.0,3.0,7.0,9411.0,31.877590,74.381043,2.0,0.428571
Villalba del Alcor,462.0,3.0,7.0,3366.0,89.126560,207.961973,2.0,0.428571
Gibraleón,546.0,12.0,26.0,12737.0,94.213708,204.129701,10.0,0.461538
Ayamonte,1244.0,6.0,13.0,21104.0,28.430629,61.599697,2.0,0.461538
Huelva-Costa,15407.0,103.0,215.0,289548.0,35.572686,74.253664,81.0,0.479070
Almonte,1020.0,61.0,126.0,24507.0,248.908475,514.138817,44.0,0.484127
